In [23]:
# Code based off of https://github.com/mesosbrodleto/playerank/blob/master/playerank/models/Weighter.py
# Minor changes made, but same for the most part

from sklearn.base import BaseEstimator
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
import json
import numpy as np

class Weighter(BaseEstimator):
    """Automatic weighting of performance features

    random_state : int
        RandomState instance or None, optional, default: None
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`.

    Attributes
    ----------
    clf_: LinearSVC object
        the object of the trained classifier
    weights_ : array, [n_features]
        weights of the features computed by the classifier
    """
    def __init__(self, random_state=42):
        self.random_state_ = random_state

    def fit(self, column_names, Xtrain, ytrain, scaled=False, var_threshold = 0.001 , filename='weights.json'):
        
        X = Xtrain
        y = ytrain

        if scaled:
            X = StandardScaler().fit_transform(X)

        self.feature_names_ = column_names
        self.clf_ = LinearSVC(fit_intercept=True, dual = False,  max_iter = 50000, random_state=self.random_state_)

        self.clf_.fit(X, y)

        importances = self.clf_.coef_[0]

        sum_importances = sum(np.abs(importances))
        self.weights_ = importances / sum_importances

        ## Save the computed weights into a json file
        features_and_weights = {}
        for feature, weight in sorted(zip(self.feature_names_, self.weights_),key = lambda x: x[1]):
            features_and_weights[feature]=  weight
        json.dump(features_and_weights, open('%s' %filename, 'w'))
        ## Save the object
        #pkl.dump(self, open('%s.pkl' %filename, 'wb'))

    def get_weights(self):
        return self.weights_

    def get_feature_names(self):
        return self.feature_names_

In [24]:
import pandas as pd
from create_feature_vectors import get_tournament_feature_vectors
from sklearn.model_selection import train_test_split
from training_functions import create_final_vectors


tournaments = ["110733838935136200"]

column_names =   ["level", "shutdownValue", "totalGold", "MINIONS_KILLED", "NEUTRAL_MINIONS_KILLED", 
                      "NEUTRAL_MINIONS_KILLED_YOUR_JUNGLE", "NEUTRAL_MINIONS_KILLED_ENEMY_JUNGLE", 
                      "CHAMPIONS_KILLED", "NUM_DEATHS", "ASSISTS", "WARD_PLACED", "WARD_KILLED", "VISION_SCORE", "TOTAL_DAMAGE_DEALT",
                      "TOTAL_DAMAGE_DEALT_TO_CHAMPIONS", "TOTAL_DAMAGE_TAKEN", "TOTAL_DAMAGE_SELF_MITIGATED", "TOTAL_DAMAGE_SHIELDED_ON_TEAMMATES", 
                      "TOTAL_DAMAGE_DEALT_TO_BUILDINGS", "TOTAL_DAMAGE_DEALT_TO_TURRETS", "TOTAL_DAMAGE_DEALT_TO_OBJECTIVES", 
                      "TOTAL_TIME_CROWD_CONTROL_DEALT_TO_CHAMPIONS", "TOTAL_HEAL_ON_TEAMMATES", "RESULT"]

final_vectors_df = pd.DataFrame(columns=column_names)

for tournament in tournaments:
    feature_vectors = get_tournament_feature_vectors(tournament)
    vectors_df = create_final_vectors(feature_vectors)
    final_vectors_df = pd.concat([final_vectors_df, vectors_df])

X = final_vectors_df.loc[:, final_vectors_df.columns != "RESULT"].values
y = final_vectors_df["RESULT"].values

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)

Games loaded: 1
Games loaded: 2
Games loaded: 3
Games loaded: 4
Games loaded: 5
Games loaded: 6
Games loaded: 7
Games loaded: 8
Games loaded: 9
Games loaded: 10
Game 10 could not be loaded, continuing...
Games loaded: 11
Games loaded: 12
Games loaded: 13
Game 13 could not be loaded, continuing...
Games loaded: 14
Games loaded: 15
Games loaded: 16
Games loaded: 17
Games loaded: 18
Games loaded: 19
Games loaded: 20
Game 20 could not be loaded, continuing...
Game 20 could not be loaded, continuing...
Games loaded: 21
Games loaded: 22
Games loaded: 23
Games loaded: 24
Games loaded: 25
Games loaded: 26
Games loaded: 27
Games loaded: 28
Games loaded: 29
Games loaded: 30
Games loaded: 31
Games loaded: 32
Games loaded: 33
Game 33 could not be loaded, continuing...
Games loaded: 34
Games loaded: 35
Game 35 could not be loaded, continuing...
Game 35 could not be loaded, continuing...
Games loaded: 36
Games loaded: 37
Games loaded: 38
Games loaded: 39
Games loaded: 40
Game 40 could not be loaded,

In [20]:
weighter = Weighter()
weighter.fit(column_names=column_names[:-1], Xtrain=Xtrain, ytrain=ytrain, scaled=False)

In [29]:
weighter.clf_.score(Xtest, ytest)

0.8857142857142857

In [1]:
from training_functions import train_model

model = train_model(["110733838935136200"], False)

/Users/sam/opt/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Games loaded: 1
Games loaded: 2
Games loaded: 3
Games loaded: 4
Games loaded: 5
Games loaded: 6
Games loaded: 7
Games loaded: 8
Games loaded: 9
Games loaded: 10
Game 10 could not be loaded, continuing...
Games loaded: 11
Games loaded: 12
Games loaded: 13
Game 13 could not be loaded, continuing...
Games loaded: 14
Games loaded: 15
Games loaded: 16
Games loaded: 17
Games loaded: 18
Games loaded: 19
Games loaded: 20
Game 20 could not be loaded, continuing...
Game 20 could not be loaded, continuing...
Games loaded: 21
Games loaded: 22
Games loaded: 23
Games loaded: 24
Games loaded: 25
Games loaded: 26
Games loaded: 27
Games loaded: 28
Games loaded: 29
Games loaded: 30
Games loaded: 31
Games loaded: 32
Games loaded: 33
Game 33 could not be loaded, continuing...
Games loaded: 34
Games loaded: 35
Game 35 could not be loaded, continuing...
Game 35 could not be loaded, continuing...
Games loaded: 36
Games loaded: 37
Games loaded: 38
Games loaded: 39
Games loaded: 40
Game 40 could not be loaded,